In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

In [10]:
url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"
embed = hub.Module(url)

In [11]:
embedding = embed(["The quick brown fox jumps over the lazy dog."])
print(embedding)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0712 09:36:10.209649 4572095936 saver.py:1483] Saver not created because there are no variables in the graph to restore


Tensor("module_2_apply_default/Encoder_en/hidden_layers/l2_normalize:0", shape=(1, 512), dtype=float32)


In [ ]:
data = pd.read_csv('spam.csv', encoding='latin-1')

y = list(data['v1'])
x = list(data['v2'])

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y)


In [ ]:
def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

test = encode(le, ['ham', 'spam', 'ham', 'ham'])

untest = decode(le, test)

In [ ]:
x_enc = x
y_enc = encode(le, y)


In [ ]:
x_train = np.asarray(x_enc[:5000])
y_train = np.asarray(y_enc[:5000])

x_test = np.asarray(x_enc[5000:])
y_test = np.asarray(y_enc[5000:])


In [ ]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

In [ ]:
def UniversalEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)))

input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(UniversalEmbedding, output_shape=(512, ))(input_text)
dense = Dense(256, activation='relu')(embedding)
pred = Dense(2, activation='softmax')(dense)
model = Model(inputs=[input_text], outputs=pred)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history = model.fit(x_train, y_train, epochs=1, batch_size=32)

In [ ]:
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model.load_weights('./model.h5')  
    predicts = model.predict(x_test, batch_size=32)


In [ ]:
y_test = decode(le, y_test)
y_preds = decode(le, predicts)


In [ ]:
from sklearn import metrics

metrics.confusion_matrix(y_test, y_preds)

print(metrics.classification_report(y_test, y_preds))